# CRIM Intervals:  Cadences

### What Can You Do With this Notebook?

* Find **cadences**, reporting type, tone, cadential voice functions (roles) and other information about the piece
* Count and analyze distribution of cadences

#### View Score Excerpts with Verovio in the NB


* It is also possible to display the results of the Cadence
    classifier in the Notebook with Verovio. Each excerpt is
    two measures long:  the measure of the final tone of the cadence
    and the previous measure.

* The function also displays metadata about each excerpt, drawn from the
    cadence results dataframe:  piece ID, composer, title, measures, type of
    cadence, beat of the bar in which the final tone is heard, and evaded
    status.
    
* View the cadence table:
   
> `piece.cadences()`
    
* To show the cadences with Verovio, then just:

>`piece.verovioCadences()`

* Note that pink warning messages in the output can be ignored!
   
* If you prefer to create the cadence table, then filter it in some way before passing to Verovio:

>`cadences = piece.cadences()`

(After filtering):

> `piece.verovioCadences(cadences)`




In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
import glob as glob
import os
from IPython.display import SVG
from pathlib import Path
import verovio
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


In [13]:
def verovio_print_piece(piece):
    # check path and import MEI or XML

    if piece.path.startswith('Music_Files/'):
        text_file = open(piece.path, "r")
        fetched_mei_string = text_file.read()
    else:
        response = requests.get(piece.path)
        fetched_mei_string = response.text
    # start the verovio toolkit and load the file there
    tk = verovio.toolkit()
    tk.loadData(fetched_mei_string)
    tk.setScale(30)
    tk.setOption( "pageHeight", "1500" )
    tk.setOption( "pageWidth", "3000" )


    tk.redoLayout()
    # get the number of pages and display the music
    count = tk.getPageCount()
    for c in range(1, count + 1):
        music = tk.renderToSVG(c)
        print("File Name: ", piece.file_name)
        print(piece.metadata['composer'])
        print(piece.metadata['title'])
        display(HTML(music))

## B. Importing a Piece

### B.1 Import a Piece and Check Title

In [8]:
# Select a prefix:
# prefix = 'https://crimproject.org/mei/'
prefix = 'Music_Files/'
# just add the CRIM Piece ID here
mei_file = 'Morley_1597_074_03.musicxml'
# combine strings and import
url = prefix + mei_file
piece = importScore(url)
print(piece.metadata)

Previously imported piece detected.
{'title': 'Morley_1597_074_03', 'composer': 'Morley'}


In [23]:
notes = piece.notes()
if len(notes.columns) == 3:
    


2

In [16]:
# GET EVERYTHING AND PRINT VEROVIO
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    verovio_print_piece(piece)


Previously imported piece detected.
File Name:  Morley_1597_136_2
Morley
Morley_1597_136_2


Previously imported piece detected.
File Name:  Morley_1597_133_02
Morley
Morley_1597_133_02


Previously imported piece detected.
File Name:  Morley_1597_129_5
Morley
Morley_1597_129_5


Previously imported piece detected.
File Name:  Morley_1597_074_03
Morley
Morley_1597_074_03


Previously imported piece detected.
File Name:  Morley_1597_190-193
Morley
Morley_1597_190-193


...

File Name:  Morley_1597_190-193
Morley
Morley_1597_190-193


File Name:  Morley_1597_190-193
Morley
Morley_1597_190-193


File Name:  Morley_1597_190-193
Morley
Morley_1597_190-193


File Name:  Morley_1597_190-193
Morley
Morley_1597_190-193


Previously imported piece detected.
File Name:  Morley_1597_075_1
Morley
Morley_1597_075_1


## C. Find Cadences with Modular Analysis

Alex Morgan has built a powerful tool that identifies cadences according to the combinations of two-voice **modules** that describe the typical contrapuntal motion between the various **cadential voice functions** (**CVF**) heard in Renaissance polyphony:  **cantizans and tenorizans**, **cantizans and bassiszans**, etc.

The tool uses modular analysis to identify **conjunctions** of these pairs in order to predict cadences of various kinds. But there are many combinations, especially once we consider that voices functions (or roles) can be **displaced** (as when the tenorizans role appears in the Superius part and the cantizans appears in the Tenor part), or through **irregular** motion, and even **interrupted**, as when a voice is suddenly silent. 

You can in fact check all of the cadential voice functions (CVFs) for a given piece below.  But Alex's system also conveniently **labels** the cadences according to **type**, **tone**, **evaded** and also provides information about the relative place within the piece, the adjacent cadences, and many other features, too.

Note:  **Measure** and **Beat** columns are in the body of the table, not at the Index.

**Column Headings Explained**:

* The **Key** column is the string used by the classifier to determine the label. "BC1" for instance, means "bassus, cantus, and one leading tone".  Note that these letters appear in alphabetical order, not the order of the voices in the score.
* The **CadType** is a high-level label.  **Clausula Vera** is for cadences involving only Cantizans and Tenorizans; **Authentic** is for Cantizans and Bassizans (and possibly the Tenorizans, too).  **Phrygian Clausula Vera** is like **Clausula Vera** but with the half-step motion in the downward-moving (Tenorizans) part.  **Phrygian** corresponds to **Authentic**, except that the Bassizans of course moves up a fifth or down a fourth, as is normally the case when the Tenorizans descends by half=step.  **Altizans Only** is in cases where the Cantizans is missing and the Altizans role moves to a fifth above the lowest voice.  See `print(piece.cvfs.__doc__)` for other labels.
* ** Leading Tones** is the count of leading tones motions
* **CVFs** are the **Cadential Voice Functions**, and are listed in order from top to bottom as they appear in the score.  See `print(importedPiece.cvfs.__doc__)` for details.
* The **Low** and **Tone** columns give the pitches of the **lowest sounding pitch (in any voice) at the perfection**, and the **goal tone of the cantizans** (or altizans if there is no cantizans) respectively.

* **RelLow** is the lowest pitch of each cadence shown as an interval measured against the last pitch in the **Low** column. Likewise, **RelTone** is the cadential tone shown as an interval measured against the last pitch in the **Tone** column.

* The **SinceLast** and **ToNext** columns are the time in quarter notes since the last or to the next cadence.

* The **Progress** column is a relative indication of position in the piece.  **0** is the beginning of the piece; **1.0** is the end of the piece.
* **Sounding** is the number of voices heard at the end of the cadence.



Read more via the documentation: **`print(ImportedPiece.cadences.__doc__)`** and especially **print(ImportedPiece.cvfs.__doc__)** for the voice labels

View the **Cadential Voice Function** and **Cadence Label** tables here:  https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/.  These can easily be updated with revised or new cadence types.

### Cadential Voice Functions--What they Mean

In [5]:
print(ImportedPiece.cvfs.__doc__)


        Return a dataframe of cadential voice functions in the piece. If
        `keep_keys` is set to True, the ngrams that triggered each CVF pair
        will be shown in additional columns in the table.

        Each CVF is represented with a single-character label as follows:

        Realized Cadential Voice Functions:
        "C": cantizans motion up a step (can also be ornamented e.g. Landini)
        "T": tenorizans motion down a step (can be ornamented with anticipations)
        "B": bassizans motion up a fourth or down a fifth
        "A": altizans motion, similar to cantizans, but cadences to a fifth
            above a tenorizans instead of an octave
        "L": leaping contratenor motion up an octave at the perfection
        "P": plagal bassizans motion up a fifth or down a fourth
        "Q": quintizans, like a tenorizans, but resolves down by fifth or up by
            fourth to a fourth below the goal tone of a cantizans or an octave
            below the goal tone

## Select Number of Voices and Import

In [28]:
number_voices = 3
corpus_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    notes = piece.notes()
    if len(notes.columns) == number_voices:
        corpus_list.append(name)
        print(name  + 'has been added to your list')
    else:
        print('sorry, ' + name  + 'is not a three-voice piece')

corpus = CorpusBase(corpus_list)       
        
        


Previously imported piece detected.
sorry, Music_Files/Morley_1597_132_06.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_132_07.musicxmlis not a three-voice piece
Previously imported piece detected.
Music_Files/Morley_1597_127_2.musicxmlhas been added to your list
Previously imported piece detected.
Music_Files/Morley_1597_127_3.musicxmlhas been added to your list
Previously imported piece detected.
sorry, Music_Files/Morley_1597_131_4.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_131_5.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_074_01.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_133_04.musicxmlis not a three-voice piece
Previously imported piece detected.
sorry, Music_Files/Morley_1597_133_05.musicxmlis not a three-voice piece
Previously imported piece detected.

### Classify the Cadences as Corpus

HEre is what the code should look like:

>`func = ImportedPiece.cadences
list_of_dfs = corpus.batch(func=func, kwargs={'keep_keys': True}, metadata=True)
cadence_results = pd.concat(list_of_dfs, ignore_index=False)
cadence_results['Validation'] = ""
cadence_results['Comments'] = ""
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'Pattern', 'Key', 'CadType', 'Tone','CVFs',
                'LeadingTones', 'Sounding', 'Low','RelLow','RelTone',
                'Progress','SinceLast','ToNext', 'Validation', 'Comments']
cadence_results = combined_df[col_list]
cadence_results`

In [41]:

func = ImportedPiece.cadences
list_of_dfs = corpus.batch(func=func, kwargs={'keep_keys': True}, metadata=True)
cadence_results = pd.concat(list_of_dfs, ignore_index=False)


cadence_results['Validation'] = ""
cadence_results['Comments'] = ""
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'Pattern', 'Key', 'CadType', 'Tone','CVFs',
                'LeadingTones', 'Sounding', 'Low','RelLow','RelTone',
                'Progress','SinceLast','ToNext', 'Validation', 'Comments']
cadence_results = cadence_results[col_list]
cadence_results

,Composer,Title,Measure,Beat,Pattern,Key,CadType,Tone,CVFs,LeadingTones,Sounding,Low,RelLow,RelTone,Progress,SinceLast,ToNext,Validation,Comments
24.0,Morley,Morley_1597_127_3,4,1.000000,BC1,^BC(1|3)x?z?$,Authentic,G,tCB,1.0,3.0,G2,-P4,P5,0.428571,24.0,20.0,,
44.0,Morley,Morley_1597_127_3,6,3.000000,C1T-2,^C(1|3)T-2x?z?$,Clausula Vera,C,CT,1.0,3.0,C3,P1,P8,0.785714,20.0,12.0,,
56.0,Morley,Morley_1597_127_3,8,1.000000,BC1,^BC(1|3)x?z?$,Authentic,C,CB,1.0,3.0,C3,P1,P8,1.000000,12.0,8.0,,
16.0,Morley,Morley_1597_129_6,3,1.000000,BC1,^BC(1|3)x?z?$,Authentic,G,CB,1.0,3.0,G3,P1,P8,1.000000,16.0,8.0,,
8.0,Morley,Morley_1597_128_06,2,1.000000,c-1z,c-1z,NaN,NaN,cz,NaN,2.0,G3,P8,NaN,0.333333,8.0,16.0,,
24.0,Morley,Morley_1597_128_06,4,1.000000,BC1,^BC(1|3)x?z?$,Authentic,G,tCB,1.0,3.0,G2,P1,P8,1.000000,16.0,8.0,,
8.0,Morley,Morley_1597_128_07,2,1.000000,BC1,^BC(1|3)x?z?$,Authentic,G,tCB,1.0,3.0,G2,P1,P8,1.000000,8.0,8.0,,
12.0,Morley,Morley_1597_129_1,2,3.000000,C1T-2,^C(1|3)T-2x?z?$,Clausula Vera,C,CT,1.0,3.0,C3,P1,P8,1.000000,12.0,4.0,,
8.0,Morley,Morley_1597_128_11,1,2.333333,C1z,^C(1|3)z$,Abandoned Clausula Vera,G,zC,1.0,2.0,G3,P1,P8,0.333333,8.0,16.0,,
24.0,Morley,Morley_1597_128_11,3,3.000000,C1t2,^C(1|3)t(1|2)x?z?$,Evaded Clausula Vera,G,tC,1.0,3.0,G3,P1,P8,1.000000,16.0,4.0,,


### Summarize by Tone, Type, and Functions

* You can easily summarize the data in various ways:

    >`cadences.groupby(['Tone', 'CadType', 'CVFs']).size().reset_index(name='counts')`

In [42]:
cadence_summary = cadence_results.groupby(['Title', 'Tone', 'CadType', 'CVFs']).size().reset_index(name='counts')
cadence_summary.to_csv('Morley_a3_Cad_Summary_Table.csv')
cadence_summary

,Title,Tone,CadType,CVFs,counts
0,Morley_1597_068,A,Authentic,TCB,1
1,Morley_1597_068,D,Clausula Vera,CT,1
2,Morley_1597_068,G,Authentic,TCB,2
3,Morley_1597_068,G,Authentic,tCB,1
4,Morley_1597_068,G,Clausula Vera,CT,2
5,Morley_1597_127_3,C,Authentic,CB,1
6,Morley_1597_127_3,C,Clausula Vera,CT,1
7,Morley_1597_127_3,G,Authentic,tCB,1
8,Morley_1597_127_4,G,Evaded Clausula Vera,tC,1
9,Morley_1597_127_7,G,Evaded Clausula Vera,tC,1


### C.4.  Display Results with Verovio

#### For the Corpus
>`number_voices = 3
corpus_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    notes = piece.notes()
    if len(notes.columns) == number_voices:
        piece.verovioCadences()`
        
#### For Just One Piece

>`prefix = 'Music_Files/'
music_file = 'Morley_1597_074_03.musicxml'
url = prefix + music_file
piece = importScore(url)
print(piece.metadata)`

In [45]:
# FOR ONE PIECE
prefix = 'Music_Files/'
music_file = 'Morley_1597_074_03.musicxml'
url = prefix + music_file
piece = importScore(url)
piece.verovioCadences()

Previously imported piece detected.
Results:
File Name:  Morley_1597_074_03
Morley
Morley_1597_074_03
Cadence End Measure: 5
Beat:  1.0
Cadence Tone:  nan
Cadence Type:  Evaded Altizans Only
Cadential Voice Functions:  aT


In [47]:
## Print the Whole Corpus according to Voice Filter
## Note you must repeat the import here
number_voices = 3
corpus_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    notes = piece.notes()
    if len(notes.columns) == number_voices:
        piece.verovioCadences()

Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_127_3
Morley
Morley_1597_127_3
Cadence End Measure: 4
Beat:  1.0
Cadence Tone:  G
Cadence Type:  Authentic
Cadential Voice Functions:  tCB


Results:
File Name:  Morley_1597_127_3
Morley
Morley_1597_127_3
Cadence End Measure: 6
Beat:  3.0
Cadence Tone:  C
Cadence Type:  Clausula Vera
Cadential Voice Functions:  CT


Results:
File Name:  Morley_1597_127_3
Morley
Morley_1597_127_3
Cadence End Measure: 8
Beat:  1.0
Cadence Tone:  C
Cadence Type:  Authentic
Cadential Voice Functions:  CB


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_129_6
Morley
Morley_1597_129_6
Cadence End Measure: 3
Beat:  1.0
Cadence Tone:  G
Cadence Type:  Authentic
Cadential Voice Functions:  CB


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_128_06
Morley
Morley_1597_128_06
Cadence End Measure: 2
Beat:  1.0
Cadence Tone:  nan
Cadence Type:  nan
Cadential Voice Functions:  cz


...

Results:
File Name:  Morley_1597_194
Morley
Morley_1597_194
Cadence End Measure: 18
Beat:  1.0
Cadence Tone:  F
Cadence Type:  Authentic
Cadential Voice Functions:  CtB


Results:
File Name:  Morley_1597_194
Morley
Morley_1597_194
Cadence End Measure: 20
Beat:  1.0
Cadence Tone:  F
Cadence Type:  Authentic
Cadential Voice Functions:  tCB


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_129_2
Morley
Morley_1597_129_2
Cadence End Measure: 2
Beat:  1.0
Cadence Tone:  C
Cadence Type:  Clausula Vera
Cadential Voice Functions:  CT


Previously imported piece detected.
Results:
File Name:  Morley_1597_129_3
Morley
Morley_1597_129_3
Cadence End Measure: 2
Beat:  3.0
Cadence Tone:  C
Cadence Type:  Evaded Clausula Vera
Cadential Voice Functions:  tC


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
